In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from sklearn import metrics
import tabulate
import itertools
import umap
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import goatools
from goatools.anno.gaf_reader import GafReader
import Bio.UniProt.GOA as GOA
import gget

# locals
import utils as ut
reload(ut)

2023-01-05 10:54:00.635032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 10:54:10.283601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-01-05 10:54:10.283670: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-05 10:54:11.321206: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-05 10:54:22.1425

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [13]:
query = ['Col1a1', 'Cd36', 'C3', 'Cd81', 'Col1a2', 'Cd36', 'Lpl', 'Lrp1', 'Cav1', 'App', 'App', 'Cav1', 'Cadm1', 'Cadm1', 'Cdh5', 'Cdh5', 'H2', 'D1', 'Aplp2', 'Aplp2', 'H2', 'D1', 'Psap', 'Lrp1', 'C3', 'C3ar1', 'Apoe', 'Lrp1', 'C3', 'Lrp1', 'Col1a1', 'Cd93']
# db = 'WikiPathways_2019_Mouse'
# db = 'KEGG_2019_Mouse'
db = 'ontology'
ef = gget.enrichr(query, database=db)

pd.set_option('display.max_colwidth', 200)


ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

Thu Jan  5 10:59:04 2023 INFO Performing Enichr analysis using database GO_Biological_Process_2021. 
    Please note that there might a more appropriate database for your application. 
    Go to https://maayanlab.cloud/Enrichr/#libraries for a full list of supported databases.
    


,path_name,p_val,overlapping_genes
0,receptor internalization (GO:0031623),9.484543e-08,"[LRP1, CD81, CAV1, CD36]"
1,positive regulation of lipid localization (GO:1905954),1.005290e-07,"[C3, LRP1, LPL]"
2,receptor-mediated endocytosis (GO:0006898),1.384648e-07,"[LRP1, CD81, CAV1, APOE, CD36]"
3,cellular response to peptide (GO:1901653),1.760348e-07,"[APP, LRP1, CAV1, CD36]"
4,receptor metabolic process (GO:0043112),1.889686e-07,"[LRP1, CD81, CAV1, CD36]"
5,positive regulation of protein binding (GO:0032092),1.889686e-07,"[APP, LRP1, CAV1, APOE]"
6,cellular response to low-density lipoprotein particle stimulus (GO:0071404),2.214016e-07,"[CD81, LPL, CD36]"
7,positive regulation of multicellular organismal process (GO:0051240),3.931158e-07,"[COL1A1, LRP1, CADM1, CAV1, APOE, CD36]"
8,purinergic nucleotide receptor signaling pathway (GO:0035590),8.058015e-07,"[C3, APP, C3AR1]"
9,positive regulation of cholesterol efflux (GO:0010875),1.071785e-06,"[LRP1, CAV1, APOE]"
